In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

/usr/local/lib/python3.5/site-packages/matplotlib/__init__.py:872: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


# Here, we develop tools to pick up data needed for calibration

In [2]:
from LeCheapEyeTracker import Client, Server, Stimulation

import numpy as np
import matplotlib.pyplot as plt
import time

import cv2
from vispy import gloo, app

## First, we need to pick up stimuli x_positions.

In [3]:
class NeoStimulation(Stimulation):
    def get_stimulus(self, t0, t):
        i = int((t-t0)//self.transition_lag)
        stim_x, stim_y = self.tabPos[i]
        return self.draw_stimulus((stim_x, stim_y)), stim_x

In [4]:
class NeoClient(Client):
    def __init__(self, et, timeline, stim_type = 'calibration'):
        Client.__init__(self, et, timeline, stim_type)
        self.stimulation = NeoStimulation(self.width//10, self.height//10, stim_type = stim_type)
        self.stims_posX = np.array([])
        self.stims_time = np.array([])
        
    def on_timer(self, event):
        if time.time()-self.start < self.timeline.max():
            image, x = self.stimulation.get_stimulus(t0 = self.start, t = time.time())
            self.stims_posX = np.append(self.stims_posX, x)
            self.stims_time = np.append(self.stims_time, time.time()-self.start)
            self.program['texture'][...] = image.astype(np.uint8).reshape((self.height//10, self.width//10, 3))
        else:
            self.close()
            
        try:
            frame = self.et.cam.grab()
            if not frame is None:
                res, t0 = self.et.process_frame(frame.copy(), self.et.clock())
                self.et.eye_pos.append([res, t0])
        except Exception as e:
            if not self.et is None: print('could not grab a frame / detect the eye position', e)
        self.update()

In [5]:
fps = 100
screen = NeoClient(et=Server(), timeline=np.linspace(0, 7., 3.*fps))
screen.app.run()

window size :  (2560, 1440)


/usr/local/lib/python3.5/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)


In [6]:
print (screen.stims_posX)

[ 0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5
  0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.33  0.33  0.33  0.33
  0.33  0.33  0.33  0.33  0.33  0.33  0.5   0.5   0.5   0.5   0.5   0.5
  0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5
  0.5   0.5   0.5   0.67  0.67  0.67  0.67  0.67  0.67  0.67  0.67  0.67
  0.67  0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5   0.5 ]


In [7]:
print (screen.stims_time)

[ 0.0080359   0.21070886  0.30880189  0.40617585  0.50434494  0.60163879
  0.6984849   0.79486299  0.89438081  0.99377394  1.09113383  1.18709993
  1.28624201  1.38180995  1.47755885  1.57289696  1.67100787  1.76659179
  1.8632338   1.95922685  2.05540895  2.15175486  2.24612093  2.34197187
  2.4405849   2.54352379  2.64248681  2.73925185  2.83481884  2.93466496
  3.03413701  3.12969685  3.22494078  3.320575    3.41613483  3.51224899
  3.60772395  3.70735097  3.80636287  3.90202785  4.00154281  4.10060191
  4.20507097  4.30140281  4.39693689  4.4955759   4.59102178  4.68790579
  4.78689981  4.88300395  4.97923589  5.07449889  5.16955686  5.264709
  5.36055779  5.45562291  5.55067301  5.65304494  5.74881482  5.84871984
  5.945086    6.04027581  6.13626695  6.23224282  6.32825398  6.42334986
  6.51855779  6.6143949   6.71331382  6.81226587  6.90771484]


## Time to pick up eye x_positions

In [8]:
print (screen.et.eye_pos)

[[(60.0, 91.0), 527107.101076883], [(61.0, 92.0), 527107.301924379], [(61.0, 93.0), 527107.400197068], [(62.0, 92.0), 527107.497111397], [(59.0, 91.0), 527107.595346874], [(61.0, 90.0), 527107.69248713], [(59.0, 93.0), 527107.789348174], [(63.0, 92.0), 527107.885886786], [(61.0, 94.0), 527107.986363972], [(61.0, 93.0), 527108.085654458], [(62.0, 92.0), 527108.18204646], [(63.0, 91.0), 527108.280793409], [(61.0, 90.0), 527108.377097745], [(61.0, 90.0), 527108.473094574], [(62.0, 93.0), 527108.568687138], [(61.0, 93.0), 527108.665955705], [(63.0, 92.0), 527108.762449343], [(62.0, 93.0), 527108.858486276], [(63.0, 94.0), 527108.954819818], [(63.0, 94.0), 527109.050643476], [(62.0, 93.0), 527109.146372659], [(64.0, 94.0), 527109.242774634], [(61.0, 92.0), 527109.337291108], [(61.0, 92.0), 527109.43516413], [(63.0, 96.0), 527109.534558425], [(61.0, 96.0), 527109.635550403], [(60.0, 93.0), 527109.734530189], [(61.0, 93.0), 527109.830342191], [(59.0, 93.0), 527109.928564231], [(60.0, 95.0), 5

Good !

In [9]:
! git commit -am 'We have'

[master 7c57ff3] have to fix processing frame
 1 file changed, 77 insertions(+), 5 deletions(-)


In [10]:
! git push

Counting objects: 4, done.
Delta compression using up to 8 threads.
Compressing objects: 100% (4/4), done.
Writing objects: 100% (4/4), 1.38 KiB | 0 bytes/s, done.
Total 4 (delta 3), reused 0 (delta 0)
To https://github.com/meduz/LeCheapEyeTracker
   38d8acc..7c57ff3  master -> master
